In [1]:
#Connectings to Milvus and Redis

import redis
import milvus

milv = milvus.Milvus(host = '127.0.0.1', port = 19530)
red = redis.Redis(host = '127.0.0.1', port=6379, db=0)

In [2]:
#Creating collection

import time

collection_name = "audio_collection"
milv.drop_collection(collection_name) 
red.flushdb()
time.sleep(.1)

collection_param = {
            'collection_name': collection_name,
            'dimension': 2048,
            'index_file_size': 1024,  # optional
            'metric_type': milvus.MetricType.L2  # optional
            }

status, ok = milv.has_collection(collection_name)

if not ok:
    status = milv.create_collection(collection_param)
    print(status)

Status(code=0, message='Create collection successfully!')


In [3]:
#Indexing collection

index_param = {
    'nlist': 512
}

status = milv.create_index(collection_name, milvus.IndexType.IVF_SQ8, index_param)
status, index = milv.get_index_info(collection_name)

In [4]:
import librosa
import numpy as np
from panns_inference import SoundEventDetection, labels

def embed_and_save(path, at):
    audio, _ = librosa.core.load(path, sr=32000, mono=True)
    audio = audio[None, :]
    try:
        _, embedding = at.inference(audio)
        embedding = embedding/np.linalg.norm(embedding)
        status, ids = milv.insert(collection_name=collection_name, records=embedding)
        if not status.OK():
            print("Insert failed: {}".format(status))
        else:
            red.set(str(ids[0]), path)
            print("success")
    except:
        print("failed: " + path)
        

In [5]:
import os
from panns_inference import AudioTagging

rootdir = './data/game_sound'
at = AudioTagging(checkpoint_path=None, device='cpu')


print("Starting Insert")
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        path = os.path.join(subdir, file)
        embed_and_save(path, at)
print("Insert Done")
        
        

Checkpoint path: /Users/filiphaltmayer/panns_data/Cnn14_mAP=0.431.pth
Using CPU.
Starting Insert
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
failed: ./data/game_sound/剑击中铁器4-YS070521.wav
success
success
success
success
success
success
success
success
success
success
success
failed: ./data/game_sound/砍在身上－xh20070521.wav
success
failed: ./data/game_sound/打到身上－xh20070521.wav
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
failed: ./data/game_sound/打在身上2－xh20070521.wav
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
success
failed: ./data